# Q-1

Load the houseprices data from Kaggle.

Reimplement your model from the previous lesson.

Try OLS, Lasso, Ridge and ElasticNet regressions using the same model specification. This time, you need to do k-fold cross-validation to choose the best hyperparameter values for your models. Which model is the best? Why?



In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

pd.options.display.max_columns = None

In [2]:
housePrices = pd.read_csv("house_train.csv")
housePrices.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
housePrices.drop(['PoolQC', "Fence", "MiscFeature","FireplaceQu","Alley","LotFrontage"],axis=1,inplace=True)

In [4]:
housePrices["GarageYrBlt"].fillna(housePrices["GarageYrBlt"].mode()[0],inplace=True)
housePrices["GarageCond"].fillna(housePrices["GarageCond"].mode()[0],inplace=True)
housePrices["GarageFinish"].fillna(housePrices["GarageFinish"].mode()[0],inplace=True)
housePrices["GarageArea"].fillna(housePrices["GarageArea"].mode()[0],inplace=True)
housePrices["GarageQual"].fillna(housePrices["GarageQual"].mode()[0],inplace=True)
housePrices["GarageCars"].fillna(housePrices["GarageCars"].mode()[0],inplace=True)
housePrices["GarageType"].fillna(housePrices["GarageType"].mode()[0],inplace=True)

housePrices.dropna(inplace=True)

In [5]:
housePrices["central_air"]=pd.get_dummies(housePrices["CentralAir"], drop_first=True)

In [6]:
def heatingquc(HeatingQC):
    if HeatingQC=="Ex":
        return 5
    elif HeatingQC=="Gd":
        return 4
    elif HeatingQC=="TA":
        return 3
    elif HeatingQC=="Fa":
        return 2
    elif HeatingQC=="Po":
        return 1


housePrices["HeatingQc"] = housePrices["HeatingQC"].apply(heatingquc)   

In [7]:
def Garagequal(GarageQual):
    if GarageQual=="Ex":
        return 6
    elif GarageQual=="Gd":
        return 5
    elif GarageQual=="TA":
        return 4
    elif GarageQual=="Fa":
        return 3
    elif GarageQual=="Po":
        return 2
    elif GarageQual=="NA":
        return 1


housePrices["GarageQuall"] = housePrices["GarageQual"].apply(Garagequal) 

In [8]:
def kitchenqual(KitchenQual):
    if KitchenQual=="Ex":
        return 4
    elif KitchenQual=="Gd":
        return 3
    elif KitchenQual=="TA":
        return 2
    elif KitchenQual=="Fa":
        return 1

housePrices["Kitchen_Qual"]=housePrices["KitchenQual"].apply(kitchenqual)

In [9]:
Y = housePrices["SalePrice"]

X = housePrices[["HeatingQc","GarageQuall","GarageCars",
                 "central_air","Kitchen_Qual","GrLivArea","LotArea","OverallQual"]]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))

The number of observations in training set is 1129
The number of observations in test set is 283


In [10]:
X_test


,HeatingQc,GarageQuall,GarageCars,central_air,Kitchen_Qual,GrLivArea,LotArea,OverallQual
831,5,4,2,1,3,1200,3180,7
294,3,4,2,1,2,1644,9600,6
391,5,4,2,1,3,1961,12209,6
63,4,4,2,1,2,1710,10300,7
160,3,4,2,1,2,1232,11120,6
...,...,...,...,...,...,...,...,...
254,3,4,1,1,2,1314,8400,5
375,4,4,0,0,1,904,10020,1
1334,3,4,2,1,2,1365,2368,5
1390,5,4,2,1,3,1525,9100,7


In [11]:
X_train = sm.add_constant(X_train)

results = sm.OLS(y_train, X_train).fit()

display(results.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     437.1
Date:                Thu, 17 Sep 2020   Prob (F-statistic):               0.00
Time:                        15:19:46   Log-Likelihood:                -13548.
No. Observations:                1129   AIC:                         2.711e+04
Df Residuals:                    1120   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.757e+05   1.94e+04     -9.080      0.000   -2.14e+05   -1.38e+05
HeatingQc     2300.8279   1491.217      1.543      0.123    -625.065    5226.721
GarageQuall   1.194e+04   4823.934      2.475      0.013    2476.399    2.14e+04
GarageCars    1.705e+04   2076.967      8.210      0.000     1.3e+04    2.11e+04
central_air   1.475e+04   5473.631      2.695      0.007    4012.804    2.55e+04
Kitchen_Qual  2.118e+04   2531.179      8.367      0.000    1.62e+04    2.61e+04
GrLivArea       47.2585      2.845     16.609      0.000      41.676      52.841
LotArea          0.7785      0.112      6.977      0.000       0.560       0.997
OverallQual   2.009e+04   1418.604     14.165      0.000    1.73e+04    2.29e+04
==============================================================================
Omnibus:                      339.190   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            17806.705
Skew:                           0.549   Prob(JB):                         0.00
Kurtosis:                      22.425   Cond. No.                     2.59e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.59e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
X_test = sm.add_constant(X_test)

results = sm.OLS(y_test, X_test).fit()

display(results.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.823
Method:                 Least Squares   F-statistic:                     164.9
Date:                Thu, 17 Sep 2020   Prob (F-statistic):          4.60e-100
Time:                        15:19:46   Log-Likelihood:                -3334.1
No. Observations:                 283   AIC:                             6686.
Df Residuals:                     274   BIC:                             6719.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         -1.91e+05   3.41e+04     -5.609      0.000   -2.58e+05   -1.24e+05
HeatingQc     5641.0166   2299.785      2.453      0.015    1113.522    1.02e+04
GarageQuall   1.355e+04   8494.135      1.596      0.112   -3167.477    3.03e+04
GarageCars    1.316e+04   3346.214      3.932      0.000    6569.670    1.97e+04
central_air  -1826.3584   8737.309     -0.209      0.835    -1.9e+04    1.54e+04
Kitchen_Qual  2.173e+04   4244.971      5.118      0.000    1.34e+04    3.01e+04
GrLivArea       47.3713      5.505      8.605      0.000      36.533      58.209
LotArea          2.1152      0.401      5.273      0.000       1.325       2.905
OverallQual   2.069e+04   2359.328      8.769      0.000     1.6e+04    2.53e+04
==============================================================================
Omnibus:                      102.426   Durbin-Watson:                   1.877
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              472.533
Skew:                           1.432   Prob(JB):                    2.46e-103
Kurtosis:                       8.646   Cond. No.                     2.11e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
lrm = LinearRegression()
lrm.fit(X_train, y_train)

LinearRegression()

In [14]:
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

In [15]:
print("R-squared of the model in training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.7574137903245499
-----Test set statistics-----
R-squared of the model in test set is: 0.8171240677964811
Mean absolute error of the prediction is: 23945.00007715092
Mean squared error of the prediction is: 1065224366.9198776
Root mean squared error of the prediction is: 32637.775152725677
Mean absolute percentage error of the prediction is: 14.57800379144466


In [28]:
from sklearn.linear_model import Ridge

# Fitting a ridge regression model. Alpha is the regularization
# parameter (usually called lambda). As alpha gets larger, parameter
# shrinkage grows more pronounced.
ridgeregr = Ridge(alpha=10**4) 
ridgeregr.fit(X_train, y_train)


y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.5714781625499736
-----Test set statistics-----
R-squared of the model in test set is: 0.6498355877460962
Mean absolute error of the prediction is: 30520.833393738983
Mean squared error of the prediction is: 2039654206.3606691
Root mean squared error of the prediction is: 45162.53100038426
Mean absolute percentage error of the prediction is: 17.76170987089457


In [23]:
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=10*4) 
lassoregr.fit(X_train, y_train)


y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.757405559826172
-----Test set statistics-----
R-squared of the model in test set is: 0.8173984375471592
Mean absolute error of the prediction is: 23907.35811293845
Mean squared error of the prediction is: 1063626205.0379583
Root mean squared error of the prediction is: 32613.282647380933
Mean absolute percentage error of the prediction is: 14.548685989843671


In [20]:
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet(alpha=10**5, l1_ratio=0.5) 
elasticregr.fit(X_train, y_train)

y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.4859363439235538
-----Test set statistics-----
R-squared of the model in test set is: 0.5508224666943221
Mean absolute error of the prediction is: 35033.09115868444
Mean squared error of the prediction is: 2616390510.1393447
Root mean squared error of the prediction is: 51150.66480642597
Mean absolute percentage error of the prediction is: 20.804949413270617


R-square değerlerine bakıldığında en yüksek başarı OLS metodunda olduğu için OLS tercih edilir. 